In [16]:
pip install unidecode scikit-learn BeautifulSoup4 pandas requests lxml selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 29.9 kB/s eta 0:00:0000:0100:10m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 122.3 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.preprocessing import StandardScaler
from requests.auth import HTTPBasicAuth
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import numpy as np
import unidecode
import requests
import locale
import pickle
import re 
import lxml
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [4]:


driver = webdriver.Chrome()

try:
    # Load the page
    driver.get('https://www.senscritique.com/films/tops/top111')

    # Scroll down to load all content (you might need to adjust the range or use a more sophisticated scroll strategy)
    for _ in range(8):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # wait for content to load

    # Now the page should be fully loaded
    page_source = driver.page_source

finally:
    # Always close the driver to clean up
    driver.quit()

# Now you can use BeautifulSoup to parse the HTML
soup = BeautifulSoup(page_source, 'html.parser')

print(soup)


<html lang="fr"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><meta content="index,follow,max-snippet:-1" name="robots"/><meta content="SensCritique" name="author"/><meta content="France" name="country"/><meta content="48.8685823;2.3480648" name="geo.position"/><meta content="FR" name="geo.country"/><meta content="48.8685823,2.3480648" name="ICBM"/><meta content="127524802986" property="fb:pages"/><link href="/app-icons/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/><link href="/app-icons/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/><link href="/app-icons/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/><link href="/app-icons/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/><link href="/app-icons/apple-touch-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/><link href="/app-icons/apple-touch-icon-120x120.png" rel="apple-touch-icon" sizes="120x120"/><link href="/app-icons

In [5]:
import regex as re

def film_info(link):

    # combine them to the new url
    base_url = 'https://www.senscritique.com'
    film_url = base_url + link

    # convert the url content to soup object
    film_page = requests.get(film_url)
    film_page_soup = BeautifulSoup(film_page.content, "html.parser")

    # use a new dict to save info
    extrainfo_dict = {}

    extrainfo_dict['film_year'] = film_page_soup.find('p', {'class': 'Text__SCTitle-sc-1aoldkr-1 CoverProductInfos__StyledText-sc-1un0kh1-13 eGhlHy jugtWW'}).text

    stats_elements = film_page_soup.select('p.Text__SCText-sc-1aoldkr-0.Stats__Text-sc-1u6v943-2.gATBvI')
    if len(stats_elements) == 3:
        extrainfo_dict['stars'] = stats_elements[0].text
        extrainfo_dict['saves'] = stats_elements[1].text
        extrainfo_dict['favorites'] = stats_elements[2].text
    
    critique_number_text = film_page_soup.select('.NavigationTab__WrapperTextStyled-sc-18dtd9d-7')[2].text
    extrainfo_dict['critique_number'] = int(critique_number_text.split('(')[-1].split(')')[0])

    return extrainfo_dict



In [6]:
def find_text(element, search_criteria):
    found_element = element.find(**search_criteria)
    return found_element.text if found_element else None

# Define a dictionary to map the keys to the search criteria
search_criteria_dict = {
    'original_title': {'name': 'p', 'attrs': {"class": "ProductListItem__OriginalTitle-sc-1jkxxpj-4 gATBvH"}},
    'rating': {'name': 'div', 'attrs': {"data-testid": "Rating"}},
    'director': {'name': 'a', 'attrs': {"data-testid": "link"}},
    'duration': {'name': 'span', 'attrs': {"data-testid": "duration"}},
    'genres': {'name': 'span', 'attrs': {"data-testid": "genres"}},
    'ranking': {'name': 'span', 'attrs':{'data-testid': 'product-title-wrapper'}}
}

all_films_dict = {}
films = soup.find_all('div', class_="ProductListItem__Wrapper-sc-1jkxxpj-1 kusRkg")

for film in films:
    title = find_text(film, {'name': 'a', 'attrs': {"data-testid": "product-title"}})
    new_film_dict = {key: find_text(film, criteria) for key, criteria in search_criteria_dict.items()}

    link = film.find('a', {"data-testid": "product-title"})['href']

    new_film_dict.update(film_info(link))
    
    all_films_dict[title] = new_film_dict


print(all_films_dict)


{'Douze Hommes en colère (1957)': {'original_title': None, 'rating': '8.7', 'director': 'Sidney Lumet', 'duration': '1 h 36 min. ', 'genres': 'Policier, Drame', 'ranking': '1. Douze Hommes en colère (1957)', 'film_year': '1957', 'stars': '51.7K', 'saves': '16.3K', 'favorites': '6.8K', 'critique_number': 592}, 'Harakiri (1962)': {'original_title': None, 'rating': '8.6', 'director': 'Masaki Kobayashi', 'duration': '2 h 13 min. ', 'genres': 'Drame', 'ranking': '2. Harakiri (1962)', 'film_year': '1962', 'stars': '8.2K', 'saves': '12.7K', 'favorites': '1.5K', 'critique_number': 114}, 'Blade Runner : The Final Cut (2007)': {'original_title': None, 'rating': '8.6', 'director': 'Ridley Scott', 'duration': '1 h 57 min. ', 'genres': 'Science-fiction', 'ranking': '3. Blade Runner : The Final Cut (2007)', 'film_year': '2007', 'stars': '1.2K', 'saves': '268', 'favorites': '111', 'critique_number': 3}, 'Le Bon, la Brute et le Truand (1966)': {'original_title': None, 'rating': '8.5', 'director': 'Ser

In [7]:
print(len(all_films_dict.keys()))

111


In [9]:
import json
with open('film_info.json', 'w') as json_file:
    json.dump(all_films_dict, json_file)


In [ ]:
df = pd.read_json('film_info.json')

print(df)